In [0]:
%sql
USE CATALOG instacart;

In [0]:
%sql
-- =========================================
-- 1) USER FEATURES FROM SILVER.DIM_USER
--    (hour peak + window, mean gap, totals)
-- =========================================
CREATE OR REPLACE TEMP VIEW v_user_base AS
SELECT
  user_sk,
  user_id,
  total_orders,
  mean_days_between_orders,
  /* peak hour from 24-bin histogram */
  ARRAY_POSITION(hour_pref_hist, ARRAY_MAX(hour_pref_hist)) - 1 AS dominant_hour,
  CASE 
    WHEN ARRAY_MAX(hour_pref_hist) IS NULL THEN 'Unknown'
    WHEN ARRAY_POSITION(hour_pref_hist, ARRAY_MAX(hour_pref_hist)) - 1 BETWEEN 6  AND 11 THEN 'Morning'
    WHEN ARRAY_POSITION(hour_pref_hist, ARRAY_MAX(hour_pref_hist)) - 1 BETWEEN 12 AND 17 THEN 'Afternoon'
    WHEN ARRAY_POSITION(hour_pref_hist, ARRAY_MAX(hour_pref_hist)) - 1 BETWEEN 18 AND 21 THEN 'Evening'
    ELSE 'Late Night'
  END AS preferred_window
FROM instacart.silver.dim_user;

-- =========================================
-- 2) DOMINANT DOW & LAST GAP (from BRONZE)
--    (mask alone can’t give a mode; derive from orders)
-- =========================================
-- 2a) dominant_dow per user (most frequent order_dow)
CREATE OR REPLACE TEMP VIEW v_dom_dow AS
WITH counts AS (
  SELECT user_id, order_dow, COUNT(*) AS c
  FROM instacart.bronze.orders
  GROUP BY user_id, order_dow
),
ranked AS (
  SELECT
    user_id, order_dow,
    ROW_NUMBER() OVER (PARTITION BY user_id ORDER BY c DESC, order_dow) AS rn
  FROM counts
)
SELECT user_id, CAST(order_dow AS INT) AS dominant_dow
FROM ranked
WHERE rn = 1;

-- 2b) last_gap_days = gap on the latest order per user
CREATE OR REPLACE TEMP VIEW v_last_gap AS
WITH w AS (
  SELECT
    user_id,
    days_since_prior_order,
    order_number,
    ROW_NUMBER() OVER (PARTITION BY user_id ORDER BY order_number DESC) AS rn
  FROM instacart.bronze.orders
)
SELECT
  user_id,
  CAST(COALESCE(days_since_prior_order, 0) AS DOUBLE) AS last_gap_days
FROM w
WHERE rn = 1;

-- =========================================
-- 3) TEMPORAL + ENGAGEMENT + SEGMENT RULES
--    (no dependency on gold.user_activity_base)
-- =========================================
CREATE OR REPLACE TEMP VIEW v_segments AS
SELECT
  ub.user_sk,
  ub.user_id,
  dd.dominant_dow,
  ub.dominant_hour,
  ub.preferred_window,
  ub.mean_days_between_orders,
  lg.last_gap_days,

  /* Temporal group (window + day) */
  CASE
    WHEN ub.preferred_window = 'Late Night' THEN 'Late-Night'
    WHEN ub.preferred_window = 'Morning'  AND dd.dominant_dow BETWEEN 1 AND 5 THEN 'Morning Weekday'
    WHEN ub.preferred_window = 'Evening'  AND dd.dominant_dow IN (0,5,6)      THEN 'Evening Weekend'
    ELSE 'Balanced'
  END AS temporal_group,

  /* Segment (cadence + dormancy) */
  CASE
    WHEN lg.last_gap_days >= 30 THEN 'Dormant Users'
    WHEN ub.mean_days_between_orders <= 7 THEN 'Weekly Shoppers'
    ELSE 'Monthly Stockers'
  END AS segment,

  /* Engagement group (risk-aware) */
  CASE
    WHEN lg.last_gap_days >= 30 THEN 'Dormant User'
    WHEN lg.last_gap_days > ub.mean_days_between_orders * 1.5 THEN 'At-Risk Regular'
    ELSE 'Active Loyalist'
  END AS engagement_group,

  /* risk flag for convenience */
  (lg.last_gap_days >= 30 OR lg.last_gap_days > ub.mean_days_between_orders * 1.5) AS risk_flag

FROM v_user_base ub
LEFT JOIN v_dom_dow dd ON dd.user_id = ub.user_id
LEFT JOIN v_last_gap lg ON lg.user_id = ub.user_id;

-- =========================================
-- 4) TOP 3 PRODUCTS PER USER (direct from BRONZE)
-- =========================================
CREATE OR REPLACE TEMP VIEW v_top3 AS
WITH prod_rank AS (
  SELECT
    o.user_id,
    p.product_name,
    COUNT(*) AS times_ordered,
    ROW_NUMBER() OVER (
      PARTITION BY o.user_id
      ORDER BY COUNT(*) DESC, p.product_name
    ) AS rn
  FROM instacart.bronze.order_products_prior op
  JOIN instacart.bronze.orders   o ON op.order_id  = o.order_id
  JOIN instacart.bronze.products p ON op.product_id = p.product_id
  GROUP BY o.user_id, p.product_name
)
SELECT
  user_id,
  MAX(CASE WHEN rn = 1 THEN product_name END) AS top_product_1,
  MAX(CASE WHEN rn = 2 THEN product_name END) AS top_product_2,
  MAX(CASE WHEN rn = 3 THEN product_name END) AS top_product_3
FROM prod_rank
WHERE rn <= 3
GROUP BY user_id;

-- =========================================
-- 5) FINAL GOLD TABLE (AI-ready)
-- =========================================
CREATE TABLE IF NOT EXISTS instacart.gold.customer_segments (
  user_sk BIGINT,
  user_id INT,
  temporal_group STRING,
  engagement_group STRING,
  segment STRING,
  risk_flag BOOLEAN,
  dominant_dow TINYINT,
  dominant_hour TINYINT,
  preferred_window STRING,
  mean_days_between_orders DOUBLE,
  last_gap_days DOUBLE,
  top_product_1 STRING,
  top_product_2 STRING,
  top_product_3 STRING,
  updated_at TIMESTAMP
)
USING DELTA
LOCATION 'abfss://processed-data@datastorage00578.dfs.core.windows.net/Instacart/gold/customer_segments';

MERGE INTO instacart.gold.customer_segments AS tgt
USING (
  SELECT
    s.user_sk,
    s.user_id,
    s.temporal_group,
    s.engagement_group,
    s.segment,
    s.risk_flag,
    s.dominant_dow,
    s.dominant_hour,
    s.preferred_window,
    s.mean_days_between_orders,
    s.last_gap_days,
    t.top_product_1,
    t.top_product_2,
    t.top_product_3,
    CURRENT_TIMESTAMP() AS updated_at
  FROM v_segments s
  LEFT JOIN v_top3 t ON t.user_id = s.user_id
) AS src
ON tgt.user_id = src.user_id
WHEN MATCHED THEN UPDATE SET
  user_sk                  = src.user_sk,
  temporal_group           = src.temporal_group,
  engagement_group         = src.engagement_group,
  segment                  = src.segment,
  risk_flag                = src.risk_flag,
  dominant_dow             = src.dominant_dow,
  dominant_hour            = src.dominant_hour,
  preferred_window         = src.preferred_window,
  mean_days_between_orders = src.mean_days_between_orders,
  last_gap_days            = src.last_gap_days,
  top_product_1            = src.top_product_1,
  top_product_2            = src.top_product_2,
  top_product_3            = src.top_product_3,
  updated_at               = src.updated_at
WHEN NOT MATCHED THEN INSERT (
  user_sk, user_id, temporal_group, engagement_group, segment, risk_flag,
  dominant_dow, dominant_hour, preferred_window,
  mean_days_between_orders, last_gap_days,
  top_product_1, top_product_2, top_product_3, updated_at
) VALUES (
  src.user_sk, src.user_id, src.temporal_group, src.engagement_group, src.segment, src.risk_flag,
  src.dominant_dow, src.dominant_hour, src.preferred_window,
  src.mean_days_between_orders, src.last_gap_days,
  src.top_product_1, src.top_product_2, src.top_product_3, src.updated_at
);


In [0]:
%sql
select * from instacart.gold.customer_segments limit 20;